In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from plotly.graph_objs import Scatter, Layout
from plotly.offline import plot

Using TensorFlow backend.


In [2]:
def load_data(df, sequence_length=10, split=0.8):
    data_all = np.array(df).astype(float)    # 轉為浮點型別矩陣
    #print(data_all.shape) # (241,1)
    scaler = MinMaxScaler()
    data_all = scaler.fit_transform(data_all)  # 將數據縮放為 0~1 之間
    data = []
    # data 資料共有 (241-10-1)=230 筆
    for i in range(len(data_all) - sequence_length - 1):
        # 每筆 data 資料有 11 欄
        data.append(data_all[i: i + sequence_length + 1])
    reshaped_data = np.array(data).astype('float64')

    x = reshaped_data[:, :-1] # 第 1至第10個欄位為 特徵
    y = reshaped_data[:, -1]  # 第 11個欄位為 label
    #print(x.shape,y.shape) # (230,10,1) (230,1)
    split_boundary = int(reshaped_data.shape[0] * split)
    train_x = x[: split_boundary] # 前 80% 為 train 的特徵
    test_x = x[split_boundary:]   # 最後 20% 為 test 的特徵
 
    train_y = y[: split_boundary] # 前 80% 為 train 的 label
    test_y = y[split_boundary:]   # 最後 20% 為 test 的 label

    return train_x, train_y, test_x, test_y, scaler

def build_model():
    model = Sequential()     
    # 隱藏層：256 個神經元，input_shape：(10,1)
    # TIME_STEPS=10,INPUT_SIZE=1
    model.add(LSTM(input_shape=(10,1),units=256,unroll=False))
    model.add(Dense(units=1)) # 輸出層：1 個神經元
    #compile:loss, optimizer, metrics
    model.compile(loss="mse", optimizer="adam", metrics=['accuracy'])
    return model

def train_model(train_x, train_y, test_x, test_y):
    model = build_model()
    try:
        model.fit(train_x, train_y, batch_size=100, epochs=300, validation_split=0.1)
        predict = model.predict(test_x)
        predict = np.reshape(predict, (predict.size, )) #轉換為1維矩陣
    except KeyboardInterrupt:
        print(predict)
        print(test_y)
    return predict, test_y #傳回 預測值和真實值


In [4]:
# 主程式
pd.options.mode.chained_assignment = None  #取消顯示pandas資料重設警告
filename = './data/Train_X.csv'
df = pd.read_csv(filename, encoding='cp950')  #以pandas讀取檔案
ddprice=pd.DataFrame(df['收盤價'])
ddprice

,收盤價
0,9.04
1,57.60
2,29.50
3,34.76
4,24.30
...,...
405205,102.20
405206,94.86
405207,98.22
405208,NaN


In [ ]:
train_x, train_y, test_x, test_y, scaler =load_data(ddprice, sequence_length=10, split=0.7)
# train_x 共 230*0.8=184 筆, test_x 共 230*0.2=46 筆
#print(train_x.shape,train_y.shape) # (184,10,1) (184,1)
#print(test_x.shape,test_y.shape)   # (46,10,1)  (46,1)
predict_y, test_y = train_model(train_x, train_y, test_x, test_y)
predict_y = scaler.inverse_transform([[i] for i in predict_y]) # 還原
test_y = scaler.inverse_transform(test_y)  # 還原



Train on 255275 samples, validate on 28364 samples
Epoch 1/300
255275/255275 [==============================] - 840s 3ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00 - E - ETA:  - ETA: 7s - loss: nan - accuracy: 0 - ETA: 4s - loss: nan - accura
Epoch 2/300
255275/255275 [==============================] - 395s 2ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00 - loss: nan - accurac - ETA: 9:00 - loss: nan - accu - ETA: 6:52 - ETA: 6:54 - loss: nan - accurac - ETA: 6:45 - loss: nan - ac - ETA: 6:35 - loss: n - ETA: 6:29 - loss: nan - accuracy:  - ETA: 6:24 - loss: nan - accuracy: 0.0000e+0 - ETA: 6:23 - loss: nan - accu - ETA: 6:15 - loss: nan - accuracy: 0.0000e+0 - ETA: 6:15 - loss: nan - accuracy: 0. - ETA: 6:11 - loss: nan - accuracy: 0.000 - ETA: 6:0 - ETA: 5:38 - loss: - ETA: 5 - ETA: 5:44 - loss: nan - accuracy: 0.0 - ETA: 5:33 - loss: nan - accuracy: 0.0 - ETA: 5:30 - loss: nan - accuracy: 0.0000e+ - ETA: 5

206900/255275 [=======================>......] - ETA: 1:03 - loss: nan - accuracy: 0.0000e+00- ETA: 3: - ETA: 4:49 - loss: nan - accuracy: 0.0 - ETA: 4:48 - ETA: 4:30 - loss: na - ETA: 6:42 - loss: - ETA: 6:25 - loss: nan - accur - ETA: 6:16 - loss: nan - accuracy - ETA: 6:09 - loss: nan  - ETA: 5:58 - loss: nan - accuracy: 0.0000e+0 - ETA: 5:57 - loss: nan - accur - ETA: 5:50 - loss: nan - accuracy: 0.0000e - ETA: 5:49 - loss: nan - accura - ETA: 5:49 - loss: nan - accuracy: 0.000 - ETA: 5:48 - loss: nan - accuracy: 0.0000e+ - ETA: 5:47 - loss: nan - accu - ETA: 5:39 - loss: nan - accuracy: 0.0000e - ETA: 5:38 - loss: nan - accuracy: 0.0000e+ - ETA: 5:37 - loss: nan - accuracy - ETA: 5 - ETA: 5:17 - loss: nan - accuracy: 0.0000e+0 - ETA: 5:17 - loss: nan - accuracy: 0.00 - ETA: 5:15 - loss: nan - accuracy: 0.0000e+ - ETA: 5:14 - loss: nan - accuracy: 0.0000e+0 - ETA: 5:14 - loss: nan - accuracy: 0 - ETA: 5:11 - loss: nan - acc - ETA: 5:06 - loss: nan - accuracy: 0.0000e+0 - ETA: 5:06 

In [ ]:
plt.plot(predict_y, 'b:') #預測
plt.plot(test_y, 'r-')    #收盤價
plt.legend(['預測', '收盤價'])
plt.show()

# 建立 DataFrame，加入 predict_y、test_y，準備以 plotly 繪圖
dd2=pd.DataFrame({"predict":list(predict_y),"label":list(test_y)})
#轉換為 numpy 陣列，並轉為 float
dd2["predict"] = np.array(dd2["predict"]).astype('float64')
dd2["label"] = np.array(dd2["label"]).astype('float64')

data = [
    Scatter(y=dd2["predict"],name='預測'),
    Scatter(y=dd2["label"],name='收盤價')
] 

plot({"data": data, "layout": Layout(title='2018年個股預測圖')},auto_open=True)